In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
inpath= os.path.dirname(os.path.dirname(os.getcwd()))+"\\Data\\Source Data\\Rev.com\\"
intermedpath= os.path.dirname(os.path.dirname(os.getcwd()))+"\\Data\\Intermediate Data\\"

In [3]:
files_dict={}
for f in os.listdir(inpath):
    with open(inpath+f, encoding='utf-8') as file:
        files_dict[f] = "".join([i for i in file.readlines() if i not in ('(APPLAUSE)\n', '\n', '(APPLAUSE)', '(BOOING)\n)', '(LAUGHTER)\n', '(CROWD CHANTING)\n')]) 
#files_df=pd.DataFrame.from_dict(kaggle_files_dict, orient='index').reset_index()

In [4]:
search_for_notdonald=re.compile(r'\n(?!Donald Trump).*\:\s\(\d{2}\:\d{2}.*\)')
search_for_donald=re.compile(r'(Donald Trump\:\s{1,3}\(\d{2}\:\d{2}.*\))')
search_for_speaker=re.compile(r'(Donald Trump\:\s{1,3}\(\d{2}\:\d{2}.*\)|\n(?!Donald Trump).*\:\s\(\d{2}\:\d{2}.*\))')
#search_for_donald=re.compile(r'\nDonald Trump: \(26\:26\)\n\(singing\)')
final_list=[]
x=[]
for n in files_dict.values():
    if re.search(search_for_speaker, n):
        x=re.findall(search_for_speaker, n)
    #x=list(set([re.search(search_for, i).group(0) for i in n if re.search(search_for, i)]))
    final_list=final_list+x
final_list=list(set(final_list))
final_list

['Donald Trump: (01:46:49)',
 'Donald Trump: (01:08:02)',
 '\nSpeaker 28: (53:13)',
 'Donald Trump: (25:59)',
 '\nMike Pence: (28:37)',
 '\nAnthony Fauci: (01:32:32)',
 '\nDr. Fauci: (01:05:58)',
 '\nMike Pence: (19:24)',
 '\nSpeaker 11: (48:37)',
 '\nMale Reporter: (01:40:47)',
 'Donald Trump: (53:24)',
 'Donald Trump: (01:06:41)',
 '\nJerome Adams: (01:15:18)',
 '\nSpeaker 4: (25:07)',
 '\nSpeaker 7: (58:57)',
 '\nSpeaker 13: (01:02:55)',
 'Donald Trump: (52:07)',
 'Donald Trump: (23:31)',
 '\nDr. Birx: (12:35)',
 'Donald Trump: (23:25)',
 'Donald Trump: (01:09:07)',
 '\nSpeaker 11: (14:50)',
 '\nSpeaker 4: (59:03)',
 '\nJournalist 16: (01:44:45)',
 'Donald Trump: (30:50)',
 '\nSpeaker 4: (43:11)',
 '\nSpeaker 24: (01:15:49)',
 '\nHarris Faulkner: (20:52)',
 '\nSpeaker 23: (01:44:12)',
 '\nBrian Moynihan: (01:46)',
 '\nDonald J Trump: (04:10)',
 '\nSpeaker 8: (01:14:39)',
 'Donald Trump: (33:04)',
 '\nSecretary Azar: (58:51)',
 'Donald Trump: (20:52)',
 '\nPrime Minister Varadkar: (0

In [5]:
files_dict2={}
for key, item in files_dict.items():
    if re.search(search_for_speaker, item):
        print(key)
        x=re.split(search_for_speaker, item)
        z=x[1:]
        #[i for i in x if i!='']
        print(len(z))
        #print(z)
        final_list=[]
        for index, elem in enumerate(z):
            if (index - 1 >= 0):
                if index % 2 == 1 and len(z) % 2 ==0:
                    prev_el = str(z[index-1])
                    curr_el = str(elem)
                    if re.search(search_for_donald, prev_el):
                        final_list.append(curr_el)
                        #final_list2=final_list
                        final_list2="".join(final_list)
    else:
        final_list2=item
    text=final_list2.replace('(Applause.)', '').replace('(Laughter.)', '').replace('(APPLAUSE)', '')
    text=text.strip()
    files_dict2[key]=text

2019.07.05 Donald Trump 4th of July Event Speech Transcript.txt
46
2019.07.23 Donald Trump Speech Transcript Turning Point USA Teen Student Summit Speech.txt
432
2019.07.30 Donald Trump Speech Transcript in Jamestown Virginia.txt
110
2019.08.05 Donald Trump Statement on Mass Shootings.txt
24
2019.08.26 Donald Trump, Emmanuel Macron G7 Press Conference & Speeches.txt
406
2019.09.11 E-Cigarette Regulation and John Bolton Discussion.txt
86
2019.09.25 Press Conference Response to Impechment Inquiry.txt
188
2019.10.03 Trump Transcript Trump Says China Should Investigate Biden.txt
52
2019.10.04 Trump Claims No Quid Pro Quo.txt
146
2019.10.16 Press Conference with Italian President.txt
306
2019.10.22 Donald Trump Cabinet Meeting Transcript Trump Calls Emoluments Clause of the Constitution Phony.txt
296
2019.10.23 Donald Trump Syria Press Conference Transcript Trump Orders All Turkey Sanctions Lifted.txt
40
2019.10.27 ISIS Leader Abu Bakr al-Baghdadi Killed.txt
308
2019.12.03 Macron Trump NATO

In [6]:
files_df=pd.DataFrame.from_dict(files_dict2, orient='index', columns=['text']).reset_index()

In [7]:
files_df['date']=pd.to_datetime(files_df['index'].str[:10].replace('.', "/")).dt.strftime('%Y/%m/%d %H:%M:%S')

In [8]:
files_df['title']=files_df['index'].str[11:]
files_df['title']=files_df['title'].str.split(".").str[0]
files_df['candidate']='trump'
files_df['event_type']='speech'
files_df['campaign_related']=np.NaN
files_df['city']=np.NaN
files_df['state']=np.NaN
files_df['source']='https://www.rev.com/blog/transcript-category/donald-trump-transcripts/'

In [9]:
files_df=files_df[['candidate', 'date', 'title', 'event_type', 'city', 'state', 'campaign_related', 'text', 'source']]

In [10]:
files_df.to_csv(intermedpath+'rev.csv', index=False)